In [1]:
from langchain_community.agent_toolkits.github.toolkit import GitHubToolkit
from langchain_community.utilities.github import GitHubAPIWrapper
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Instantiate GitHub Toolkit
toolkit = GitHubToolkit.from_github_api_wrapper(GitHubAPIWrapper())
tools = [setattr(tool, "name", tool.mode) or tool for tool in toolkit.get_tools()]

llm = ChatOpenAI(model="gpt-4o", temperature=0)

agent_executor = create_react_agent(llm, tools)

def ai(query: str):
    # System prompt allows for broad range of instructions. But what does user input look like?
    events = agent_executor.stream(
        {"messages": [("system",
                       "You are a web server, serving HTML web pages. Do not use markdown for formatting. "
                       "All output must be in HTML format and will be displayed to an end user. "
                       "You need to start with doctype and html tags and provide the entire page"),
                      ("user", query)]},
        stream_mode="values",
    )
    data = ""
    for event in events:
        event["messages"][-1].pretty_print()
        data = event["messages"][-1]

    return data.content


In [3]:
from pathlib import Path

from dotenv import load_dotenv
from flask import Flask, request


# Load environment variables from .env
load_dotenv()

app = Flask(__name__)

@app.route('/', methods=['GET'])
def get_data():
    # Wow where is the logic?
    return ai(Path("my_app-step-3.txt").read_text())


## Goals
- Write natural language only
- Unstructured data
- Implicit algorithms
- ~~Dynamic tools~~
- ~~Human in the loop built-in~~
- ~~Consistent output~~
- ~~Iterative development~~

In [4]:
app.run(debug=False, port=5001, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit


================================ Human Message =================================

Find the most prolific author of github issues, and I want to see all of their issues and comments

I will need to see ONLY the author, title, and comments. HTML elements can be styled using twitter bootstrap css.

================================ Human Message =================================

Find the most prolific author of github issues, and I want to see all of their issues and comments

I will need to see ONLY the author, title, and comments. HTML elements can be styled using twitter bootstrap css.

================================== Ai Message ==================================
Tool Calls:
  get_issues (call_F9TBThaxIfwAf0fNOypkMfSn)
 Call ID: call_F9TBThaxIfwAf0fNOypkMfSn
  Args:
    no_input:================================== Ai Message ==================================
Tool Calls:
  get_issues (call_c62s0BFcivpzsifOHIiLACo7)
 Call ID: call_c62s0BFcivpzsifOHIiLACo7
  Args:
    no_input:

======

127.0.0.1 - - [25/Mar/2025 13:25:05] "GET / HTTP/1.1" 200 -


================================== Ai Message ==================================

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>GitHub Issues by Most Prolific Author</title>
    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
    <div class="container mt-5">
        <h1 class="mb-4">Issues by Most Prolific Author: <strong>yasonk</strong></h1>
        
        <div class="card mb-3">
            <div class="card-header">
                <h5 class="card-title">This is my second issue</h5>
            </div>
            <div class="card-body">
                <h6>Comments:</h6>
                <ul class="list-group">
                    <li class="list-group-item"><strong>yasonk:</strong> Comment 1, because I'm building a community</li>
                    <li class="list-group-item"><strong>yasonk:</strong> Comme

...

================================== Ai Message ==================================

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>GitHub Issues</title>
    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
    <div class="container mt-5">
        <h1 class="mb-4">GitHub Issues</h1>
        <div class="alert alert-warning" role="alert">
            No branches found in the repository. Please ensure the repository exists and has branches.
        </div>
    </div>
</body>
</html>
================================ Human Message =================================

Find the most prolific author of github issues, and I want to see all of their issues and comments

I will need to see ONLY the author, title, and comments. HTML elements can be styled using twitter bootstrap css.

Show only the comments by the author of the is

127.0.0.1 - - [25/Mar/2025 13:29:11] "GET / HTTP/1.1" 200 -


================================== Ai Message ==================================

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>GitHub Issues by Most Prolific Author</title>
    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
    <div class="container mt-5">
        <h2>Issues by Most Prolific Author: yasonk</h2>
        <div id="accordion">
            <div class="card">
                <div class="card-header" id="headingOne">
                    <h5 class="mb-0">
                        <button class="btn btn-link" data-toggle="collapse" data-target="#collapseOne" aria-expanded="true" aria-controls="collapseOne">
                            Issue #2: This is my second issue
                        </button>
                    </h5>
                </div>
                <div id="collapseOne" class="collapse

127.0.0.1 - - [26/Mar/2025 11:18:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2025 11:18:12] "GET /favicon.ico HTTP/1.1" 404 -


================================== Ai Message ==================================

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>GitHub Issues by Most Prolific Author</title>
    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
    <div class="container mt-5">
        <h1 class="mb-4">Issues by Most Prolific Author: <strong>yasonk</strong></h1>
        
        <div class="card mb-3">
            <div class="card-header">
                <h5 class="card-title">This is my second issue</h5>
            </div>
            <div class="card-body">
                <h6>Comments:</h6>
                <ul class="list-group">
                    <li class="list-group-item"><strong>yasonk:</strong> Comment 1, because I'm building a community</li>
                    <li class="list-group-item"><strong>yasonk:</strong> Comme